# Day 14 - Pandas exploration

In [101]:
import pandas as pd


df = pd.read_csv("ai_job_dataset.csv")

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   job_id                  15000 non-null  object 
 1   job_title               15000 non-null  object 
 2   salary_usd              15000 non-null  int64  
 3   salary_currency         15000 non-null  object 
 4   experience_level        15000 non-null  object 
 5   employment_type         15000 non-null  object 
 6   company_location        15000 non-null  object 
 7   company_size            15000 non-null  object 
 8   employee_residence      15000 non-null  object 
 9   remote_ratio            15000 non-null  int64  
 10  required_skills         15000 non-null  object 
 11  education_required      15000 non-null  object 
 12  years_experience        15000 non-null  int64  
 13  industry                15000 non-null  object 
 14  posting_date            15000 non-null

In [103]:
df.mean(numeric_only=True)

salary_usd                115348.965133
remote_ratio                  49.483333
years_experience               6.253200
job_description_length      1503.314733
benefits_score                 7.504273
dtype: float64

In [104]:
df.head()

,job_id,job_title,salary_usd,salary_currency,experience_level,employment_type,company_location,company_size,employee_residence,remote_ratio,required_skills,education_required,years_experience,industry,posting_date,application_deadline,job_description_length,benefits_score,company_name
0,AI00001,AI Research Scientist,90376,USD,SE,CT,China,M,China,50,"Tableau, PyTorch, Kubernetes, Linux, NLP",Bachelor,9,Automotive,2024-10-18,2024-11-07,1076,5.9,Smart Analytics
1,AI00002,AI Software Engineer,61895,USD,EN,CT,Canada,M,Ireland,100,"Deep Learning, AWS, Mathematics, Python, Docker",Master,1,Media,2024-11-20,2025-01-11,1268,5.2,TechCorp Inc
2,AI00003,AI Specialist,152626,USD,MI,FL,Switzerland,L,South Korea,0,"Kubernetes, Deep Learning, Java, Hadoop, NLP",Associate,2,Education,2025-03-18,2025-04-07,1974,9.4,Autonomous Tech
3,AI00004,NLP Engineer,80215,USD,SE,FL,India,M,India,50,"Scala, SQL, Linux, Python",PhD,7,Consulting,2024-12-23,2025-02-24,1345,8.6,Future Systems
4,AI00005,AI Consultant,54624,EUR,EN,PT,France,S,Singapore,100,"MLOps, Java, Tableau, Python",Master,0,Media,2025-04-15,2025-06-23,1989,6.6,Advanced Robotics


In [105]:
import altair as alt
import vegafusion

df_5000 = df.head(5000)

alt.Chart(df_5000).mark_point().encode(
    x="years_experience",
    y="salary_usd",
    color="education_required"
).interactive()

alt.Chart(...)

Let's calculate mean salary for each educational level required 

In [106]:
df_mean_salaries = df_5000.groupby(["years_experience", "education_required"])["salary_usd"].mean()

In [107]:
df_mean_salaries = pd.DataFrame(df_mean_salaries)

In [108]:
df_mean_salaries = df_mean_salaries.reset_index()

In [109]:
alt.Chart(df_mean_salaries).mark_line().encode(
    x="years_experience",
    y="salary_usd",
    color="education_required"
).interactive()

alt.Chart(...)

In [171]:
skills = df["required_skills"].to_frame()

In [172]:
skills

,required_skills
0,"[Tableau, PyTorch, Kubernetes, Linux, NLP]"
1,"[Deep Learning, AWS, Mathematics, Python, ..."
2,"[Kubernetes, Deep Learning, Java, Hadoop, ..."
3,"[Scala, SQL, Linux, Python]"
4,"[MLOps, Java, Tableau, Python]"
...,...
14995,"[Java, Kubernetes, Azure]"
14996,"[Mathematics, Docker, SQL, Deep Learning]"
14997,"[Scala, Spark, NLP]"
14998,"[Java, Computer Vision, Python, TensorFlow]"


In [210]:
skills_exploded = skills.explode("required_skills").value_counts().to_frame()

In [211]:
skills_exploded.head()

,count
required_skills,
Python,2959
SQL,2445
TensorFlow,2442
Kubernetes,2146
Scala,2017


Clearly, the skills haven't been exploded properly as the skills are nested within eachother thus producing duplicates


In [212]:
skills_exploded.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 48 entries, (' Python',) to ('Statistics',)
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   count   48 non-null     int64
dtypes: int64(1)
memory usage: 1.9+ KB


In [213]:
df_reset = skills_exploded.reset_index()

In [215]:
df_reset.head()

,required_skills,count
0,Python,2959
1,SQL,2445
2,TensorFlow,2442
3,Kubernetes,2146
4,Scala,2017


In [216]:
df_reset["required_skills"] = df_reset["required_skills"].str.lstrip(" ")

In [234]:
df_sorted_skills = df_reset.groupby(["required_skills"]).sum().sort_values(by=["count"], ascending=False).reset_index()

In [236]:
df_sorted_skills

,required_skills,count
0,Python,4450
1,SQL,3407
2,TensorFlow,3022
3,Kubernetes,3009
4,Scala,2794
5,PyTorch,2777
6,Linux,2705
7,Git,2631
8,Java,2578
9,GCP,2442


In [240]:
alt.Chart(df_sorted_skills).mark_bar().encode(
    x=alt.X("required_skills", sort='-y'),
    y="count"
)

alt.Chart(...)